<a href="https://colab.research.google.com/github/Nikhileswar-Komati/Suicide_Ideation/blob/master/BERT_USE_TRAIN_MULTI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

# Install java
! apt-get update -qq
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! java -version

# Install pyspark
! pip install --ignore-installed pyspark==2.4.4

# Install Spark NLP
! pip install --ignore-installed spark-nlp


openjdk version "1.8.0_275"
OpenJDK Runtime Environment (build 1.8.0_275-8u275-b01-0ubuntu1~18.04-b01)
OpenJDK 64-Bit Server VM (build 25.275-b01, mixed mode)
Processing /root/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471/pyspark-2.4.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/e3/53/c737818eb9a7dc32a7cd4f1396e787bd94200c3997c72c1dbe028587bd76/py4j-0.10.7-py2.py3-none-any.whl
Processing /root/.cache/pip/wheels/75/e4/63/ade6bdbafb87b93e87b21c22db80b606eb4d47d7aaa88e8158/spark_nlp-2.7.0-py2.py3-none-any.whl


In [ ]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version", spark.version)

Spark NLP version:  2.7.0
Apache Spark version 2.4.4


In [ ]:
import os
#kaggle username from api token 
os.environ['KAGGLE_USERNAME'] = "nikhileswarkomati"
#kaggle key from api token
os.environ['KAGGLE_KEY'] ="0ffd0be5c369d70815be7d0521aeab91"
#dataset path for downloading the data
!kaggle datasets download -d nikhileswarkomati/suicide-watch

suicide-watch.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip '/content/suicide-watch.zip'

Archive:  /content/suicide-watch.zip
replace Cleaned_Depression_Vs_Suicide.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
spark_df = spark.read.option("header", "true").option("multiLine", "true").option("quote", "\"").option("escape", "\"").option("inferSchema", "true").csv('/content/Depression_Vs_Subreddit.csv', sep = ',')

In [ ]:
spark_split_df = spark_df

(train_data, test_data, val_data) = spark_split_df.randomSplit([0.01, 0.01, 0.9], 24)

print("Train length", train_data.count())
print("Test length", test_data.count())
print("validation length", val_data.count())

Train length 6876
Test length 6875
validation length 626034


In [ ]:
from pyspark.ml import Pipeline

from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *

In [ ]:
document = DocumentAssembler()\
    .setInputCol("Text")\
    .setOutputCol("document")

use = UniversalSentenceEncoder.pretrained() \
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")

# the classes/labels/categories are in category column
sentimentdl = SentimentDLApproach()\
  .setInputCols(["sentence_embeddings"])\
  .setOutputCol("class")\
  .setLabelColumn("Subreddit")\
  .setMaxEpochs(20)\
  .setEnableOutputLogs(True)

pipeline = Pipeline(
    stages = [
        document,
        use,
        sentimentdl
    ])

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]


In [ ]:
pipelineModel = pipeline.fit(train_data)

In [ ]:
!cd ~/annotator_logs && ls -l

total 8
-rw-r--r-- 1 root root 1481 Jan  8 20:24 SentimentDLApproach_59844a7c7c32.log
-rw-r--r-- 1 root root  433 Jan  8 20:15 SentimentDLApproach_a1cfcfa2e179.log


In [ ]:
!cat ~/annotator_logs/SentimentDLApproach_59844a7c7c32.log

Training started - epochs: 20 - learning_rate: 0.005 - batch_size: 64 - training_examples: 6876
Epoch 0/20 - 2.38s - loss: 59.34515 - acc: 0.7308703 - batches: 108
Epoch 1/20 - 2.37s - loss: 57.567738 - acc: 0.76844126 - batches: 108
Epoch 2/20 - 2.18s - loss: 56.11422 - acc: 0.7745744 - batches: 108
Epoch 3/20 - 2.16s - loss: 55.052856 - acc: 0.7823139 - batches: 108
Epoch 4/20 - 2.18s - loss: 54.340836 - acc: 0.7869868 - batches: 108
Epoch 5/20 - 2.16s - loss: 53.669533 - acc: 0.7937041 - batches: 108
Epoch 6/20 - 2.25s - loss: 53.162262 - acc: 0.798377 - batches: 108
Epoch 7/20 - 2.15s - loss: 52.731255 - acc: 0.80275786 - batches: 108
Epoch 8/20 - 2.17s - loss: 52.309166 - acc: 0.8058661 - batches: 108
Epoch 9/20 - 2.23s - loss: 51.907764 - acc: 0.80951685 - batches: 108
Epoch 10/20 - 2.16s - loss: 51.536446 - acc: 0.81170726 - batches: 108
Epoch 11/20 - 2.19s - loss: 51.125496 - acc: 0.81565 - batches: 108
Epoch 12/20 - 2.13s - loss: 50.70455 - acc: 0.8195928 - batches: 108
Epoch 

In [ ]:
# actual content is inside description column
document = DocumentAssembler()\
    .setInputCol("Text")\
    .setOutputCol("document")
    
# we can also use sentence detector here 
# if we want to train on and get predictions for each sentence
# downloading pretrained embeddings
bert_sent = BertSentenceEmbeddings.pretrained('sent_small_bert_L8_512')\
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")
# the classes/labels/categories are in category column

classsifierdl = ClassifierDLApproach()\
  .setInputCols(["sentence_embeddings"])\
  .setOutputCol("class")\
  .setLabelColumn("Subreddit")\
  .setMaxEpochs(20)\
  .setEnableOutputLogs(True)

bert_sent_clf_pipeline = Pipeline(
    stages = [
        document,
        bert_sent,
        classsifierdl
    ])

sent_small_bert_L8_512 download started this may take some time.
Approximate size to download 149.1 MB
[OK!]


In [ ]:
bert_pipelineModel = bert_sent_clf_pipeline.fit(train_data)

In [ ]:
!cd ~/annotator_logs && ls -l

total 12
-rw-r--r-- 1 root root 1461 Jan  8 20:47 ClassifierDLApproach_cfbc853c01fb.log
-rw-r--r-- 1 root root 1481 Jan  8 20:24 SentimentDLApproach_59844a7c7c32.log
-rw-r--r-- 1 root root  433 Jan  8 20:15 SentimentDLApproach_a1cfcfa2e179.log


In [ ]:
# actual content is inside description column
document_assembler = DocumentAssembler().setInputCol("Text").setOutputCol("document")

tokenizer = Tokenizer().setInputCols(["document"]).setOutputCol("token")

normalizer = Normalizer().setInputCols(["token"]).setOutputCol("normalized")

stopwords_cleaner = StopWordsCleaner().setInputCols(["normalized"]).setOutputCol("cleanTokens").setCaseSensitive(False)
    
lemma = LemmatizerModel.pretrained('lemma_antbnc')\
    .setInputCols(["cleanTokens"])\
    .setOutputCol("lemma")
# we can also use sentence detector here 
# if we want to train on and get predictions for each sentence
# downloading pretrained embeddings
word_embeddings = WordEmbeddingsModel.pretrained()\
 .setInputCols(["document", "lemma"])\
 .setOutputCol("embeddings")\
 .setCaseSensitive(False)
embeddings_sentence = SentenceEmbeddings().setInputCols(["document","embeddings"]).setOutputCol("sentence_embeddings").setPoolingStrategy("AVERAGE")

# the classes/labels/categories are in category column

classsifierdl = ClassifierDLApproach()\
  .setInputCols(["sentence_embeddings"])\
  .setOutputCol("class")\
  .setLabelColumn("Subreddit")\
  .setMaxEpochs(20)\
  .setEnableOutputLogs(True)

clf_pipeline = Pipeline(
    stages = [
        document_assembler,
        tokenizer,
        normalizer,
        stopwords_cleaner,
        lemma,
        word_embeddings,
        embeddings_sentence,
        classsifierdl
    ])

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]
glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]


In [ ]:
clf_pipelineModel = clf_pipeline.fit(train_data)

In [ ]:
!cd ~/annotator_logs && ls -l

total 16
-rw-r--r-- 1 root root 1490 Jan  8 21:16 ClassifierDLApproach_56ee1fb46f28.log
-rw-r--r-- 1 root root 1461 Jan  8 20:47 ClassifierDLApproach_cfbc853c01fb.log
-rw-r--r-- 1 root root 1481 Jan  8 20:24 SentimentDLApproach_59844a7c7c32.log
-rw-r--r-- 1 root root  433 Jan  8 20:15 SentimentDLApproach_a1cfcfa2e179.log


In [ ]:
!cat ~/annotator_logs/ClassifierDLApproach_56ee1fb46f28.log

Training started - epochs: 20 - learning_rate: 0.005 - batch_size: 64 - training_examples: 6876 - classes: 2
Epoch 0/20 - 3.10s - loss: 72.29845 - acc: 0.60201097 - batches: 108
Epoch 1/20 - 2.24s - loss: 64.93205 - acc: 0.6910673 - batches: 108
Epoch 2/20 - 2.20s - loss: 64.16535 - acc: 0.69978726 - batches: 108
Epoch 3/20 - 2.24s - loss: 63.29178 - acc: 0.70869493 - batches: 108
Epoch 4/20 - 2.22s - loss: 62.46677 - acc: 0.71311754 - batches: 108
Epoch 5/20 - 2.26s - loss: 62.128746 - acc: 0.715454 - batches: 108
Epoch 6/20 - 2.24s - loss: 61.808155 - acc: 0.7182285 - batches: 108
Epoch 7/20 - 2.22s - loss: 61.563927 - acc: 0.7189586 - batches: 108
Epoch 8/20 - 2.22s - loss: 61.350502 - acc: 0.721003 - batches: 108
Epoch 9/20 - 2.21s - loss: 61.21146 - acc: 0.7218792 - batches: 108
Epoch 10/20 - 2.23s - loss: 61.10493 - acc: 0.7240696 - batches: 108
Epoch 11/20 - 2.22s - loss: 61.028774 - acc: 0.7253839 - batches: 108
Epoch 12/20 - 2.27s - loss: 60.963905 - acc: 0.725968 - batches: 1

In [ ]:
!cat ~/annotator_logs/ClassifierDLApproach_cfbc853c01fb.log

Training started - epochs: 20 - learning_rate: 0.005 - batch_size: 64 - training_examples: 6876 - classes: 2
Epoch 0/20 - 2.92s - loss: 77.26133 - acc: 0.52951854 - batches: 108
Epoch 1/20 - 2.18s - loss: 76.83232 - acc: 0.531855 - batches: 108
Epoch 2/20 - 2.18s - loss: 76.83232 - acc: 0.531855 - batches: 108
Epoch 3/20 - 2.17s - loss: 76.83232 - acc: 0.531855 - batches: 108
Epoch 4/20 - 2.17s - loss: 76.83232 - acc: 0.531855 - batches: 108
Epoch 5/20 - 2.19s - loss: 76.83232 - acc: 0.531855 - batches: 108
Epoch 6/20 - 2.18s - loss: 76.83232 - acc: 0.531855 - batches: 108
Epoch 7/20 - 2.19s - loss: 76.83232 - acc: 0.531855 - batches: 108
Epoch 8/20 - 2.17s - loss: 76.83232 - acc: 0.531855 - batches: 108
Epoch 9/20 - 2.18s - loss: 76.83232 - acc: 0.531855 - batches: 108
Epoch 10/20 - 2.18s - loss: 76.83232 - acc: 0.531855 - batches: 108
Epoch 11/20 - 2.19s - loss: 76.83232 - acc: 0.531855 - batches: 108
Epoch 12/20 - 2.21s - loss: 76.83232 - acc: 0.531855 - batches: 108
Epoch 13/20 - 2

In [ ]:
!cat ~/annotator_logs/SentimentDLApproach_59844a7c7c32.log

Training started - epochs: 20 - learning_rate: 0.005 - batch_size: 64 - training_examples: 6876
Epoch 0/20 - 2.38s - loss: 59.34515 - acc: 0.7308703 - batches: 108
Epoch 1/20 - 2.37s - loss: 57.567738 - acc: 0.76844126 - batches: 108
Epoch 2/20 - 2.18s - loss: 56.11422 - acc: 0.7745744 - batches: 108
Epoch 3/20 - 2.16s - loss: 55.052856 - acc: 0.7823139 - batches: 108
Epoch 4/20 - 2.18s - loss: 54.340836 - acc: 0.7869868 - batches: 108
Epoch 5/20 - 2.16s - loss: 53.669533 - acc: 0.7937041 - batches: 108
Epoch 6/20 - 2.25s - loss: 53.162262 - acc: 0.798377 - batches: 108
Epoch 7/20 - 2.15s - loss: 52.731255 - acc: 0.80275786 - batches: 108
Epoch 8/20 - 2.17s - loss: 52.309166 - acc: 0.8058661 - batches: 108
Epoch 9/20 - 2.23s - loss: 51.907764 - acc: 0.80951685 - batches: 108
Epoch 10/20 - 2.16s - loss: 51.536446 - acc: 0.81170726 - batches: 108
Epoch 11/20 - 2.19s - loss: 51.125496 - acc: 0.81565 - batches: 108
Epoch 12/20 - 2.13s - loss: 50.70455 - acc: 0.8195928 - batches: 108
Epoch 